Group Member: Jessica Yang, Vincent Zhu

In [2]:
# Install packages
!pip install interpret --quiet
!pip install lightgbm --quiet
!pip install shap --quiet
!pip install lime --quiet

In [3]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import shap
import requests
from lime.lime_tabular import LimeTabularExplainer
# from folktables import ACSDataSource, ACSPublicCoverage, ACSIncome, BasicProblem, adult_filter
from xgboost import XGBClassifier

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score

from interpret import show
from interpret.glassbox import LogisticRegression

In [4]:
# Define matrics

# Independence
def independence(y_hat, group):
  """
  Computes an independence metric between two specific groups.

  Args:
    y_hat (np.ndarray): Classifier predictions.
    group (np.ndarray): Array of indices corresponding to group membership.
      For this assignment, we will focus on comparing groups 1 and 2.
      These correspond to the 'White alone' and 'Black or African American'
      groups. Note that one can also compare different combinations of groups.

  Returns:
    float: independence measure
  """
  prob_a = np.sum(y_hat[group == 1])
  prob_b = np.sum(y_hat[group == 2])

  measure = prob_b / prob_a

  return measure


# Separation
def separation(y_hat, y_true, group):
  """
  Computes a separation metric between two specific groups.

  Args:
    y_hat  (np.ndarray): Classifier predictions.
    y_true (np.ndarray): Data labels.
    group  (np.ndarray): Array of indices corresponding to group membership.
      For this assignment, we will focus on comparing groups 1 and 2.
      These correspond to the 'White alone' and 'Black or African American'
      groups. Note that one can also compare different combinations of groups.

  Returns:
    float: separation true positive
    float: separation false positive
  """
  group_a = (group == 1)
  group_b = (group == 2)

  tp_a = np.sum((y_hat == 1) & (y_true == 1) & group_a)
  fp_a = np.sum((y_hat == 1) & (y_true == 0) & group_a)
  tp_b = np.sum((y_hat == 1) & (y_true == 1) & group_b)
  fp_b = np.sum((y_hat == 1) & (y_true == 0) & group_b)

  tpr_a = tp_a / np.sum((y_true == 1) & group_a)
  tpr_b = tp_b / np.sum((y_true == 1) & group_b)

  fpr_a = fp_a / np.sum((y_true == 0) & group_a)
  fpr_b = fp_b / np.sum((y_true == 0) & group_b)

  separation_true_positive = tpr_b / tpr_a if tpr_b > 0 else np.inf
  separation_false_positive = fpr_b / fpr_a if fpr_b > 0 else np.inf

  return separation_true_positive, separation_false_positive


# Sufficiency
def sufficiency(y_hat, y_true, group):
  """
  Computes a sufficiency metric between two specific groups.

  Args:
    y_hat  (np.ndarray): Classifier predictions.
    y_true (np.ndarray): Data labels.
    group  (np.ndarray): Array of indices corresponding to group membership.
      For this assignment, we will focus on comparing groups 1 and 2.
      These correspond to the 'White alone' and 'Black or African American'
      groups. Note that one can also compare different combinations of groups.

  Returns:
    float: sufficiency metric
  """
  group_a = (group == 1)
  group_b = (group == 2)

  ppv_a = np.sum((y_hat == 1) & (y_true == 1) & group_a) / np.sum((y_hat == 1) & group_a)
  ppv_b = np.sum((y_hat == 1) & (y_true == 1) & group_b) / np.sum((y_hat == 1) & group_b)

  if np.sum((y_hat == 1) & group_a) == 0:
      ppv_a = np.inf
  if np.sum((y_hat == 1) & group_b) == 0:
      ppv_b = np.inf

  sufficiency_positive = ppv_b / ppv_a if ppv_b > 0 else np.inf

  return sufficiency_positive


# SPD
def spd(sensitive_attribute, dataset, predicted_labels, majority_class, minority_class):
    """
    Calculate the Statistical Parity Difference (SPD) between majority and minority classes based on predicted labels.

    Parameters:
    - sensitive_attribute (str): Name of the column representing the sensitive attribute.
    - dataset (pd.DataFrame): The dataset containing the sensitive attribute and true outcome variable.
    - predicted_labels (pd.Series): Predicted labels for the outcome variable.
    - majority_class: Value representing the majority class in the sensitive attribute.
    - minority_class: Value representing the minority class in the sensitive attribute.

    Returns:
    - spd (float): Statistical Parity Difference between majority and minority classes.
    """

    prob_majority = np.sum(predicted_labels[dataset[sensitive_attribute] == majority_class]) / len(predicted_labels[dataset[sensitive_attribute] == majority_class])
    prob_minority = np.sum(predicted_labels[dataset[sensitive_attribute] == minority_class]) / len(predicted_labels[dataset[sensitive_attribute] == minority_class])


    spd_val = prob_minority - prob_majority

    return spd_val


# DI
def di(sensitive_attribute, dataset, predicted_labels, majority_class, minority_class):
    """
    Calculate the Disparate Impact (DI) between majority and minority classes based on predicted labels.

    Parameters:
    - sensitive_attribute (str): Name of the column representing the sensitive attribute.
    - dataset (pd.DataFrame): The dataset containing the sensitive attribute and true outcome variable.
    - predicted_labels (pd.Series): Predicted labels for the outcome variable.
    - majority_class: Value representing the majority class in the sensitive attribute.
    - minority_class: Value representing the minority class in the sensitive attribute.

    Returns:
    - di (float): Disparate Impact between majority and minority classes.
    """

    prob_majority = np.mean(predicted_labels[dataset[sensitive_attribute] == majority_class])
    prob_minority = np.mean(predicted_labels[dataset[sensitive_attribute] == minority_class])

    if prob_majority == 0:
        return np.inf

    di_val = prob_minority / prob_majority

    return di_val


# EOD
def eod(sensitive_attribute, predictions, dataset, true_labels, majority_class, minority_class):
    """
    Calculate the Equal Opportunity Difference (EOD) measure.

    Parameters:
    - sensitive_attribute: The column name of the sensitive attribute in the dataset.
    - predictions: Predictions made by the model.
    - dataset: The dataset containing the sensitive attribute and the outcome variable.
    - outcome_variable: The column name of the outcome variable in the dataset.
    - majority_class: The majority class label.
    - minority_class: The minority class label.

    Returns:
    - eod_value: The Equal Opportunity Difference measure.
    """

    majority = predictions[(dataset[sensitive_attribute] == majority_class) & (true_labels == 1)]
    minority = predictions[(dataset[sensitive_attribute] == minority_class) & (true_labels == 1)]

    tpr_majority = np.sum(majority) / len(majority)
    tpr_minority = np.sum(minority) / len(minority)

    eod_value = tpr_minority - tpr_majority

    return eod_value

# AAOD
def aaod(sensitive_attribute, predictions, dataset, true_labels, majority_class, minority_class):
    """
    Calculate the Average Absolute Odds Difference (AAOD) to measure bias.

    Parameters:
    - sensitive_attribute (str): The name of the sensitive attribute in the dataset.
    - predictions (pd.Series): The predicted values.
    - dataset (pd.DataFrame): The dataset containing the sensitive attribute, predictions, and outcome variable.
    - outcome_variable (str): The name of the outcome variable in the dataset.
    - majority_class (int): The label of the majority class.
    - minority_class (int): The label of the minority class.

    Returns:
    - float: The calculated Average Absolute Odds Difference (AAOD).
    """
    fp_majority = np.sum((predictions == 1) & (true_labels == 0) & (dataset[sensitive_attribute] == majority_class))
    tn_majority = np.sum((predictions == 0) & (true_labels == 0) & (dataset[sensitive_attribute] == majority_class))
    tp_majority = np.sum((predictions == 1) & (true_labels == 1) & (dataset[sensitive_attribute] == majority_class))
    fn_majority = np.sum((predictions == 0) & (true_labels == 1) & (dataset[sensitive_attribute] == majority_class))
    fpr_majority = fp_majority / (fp_majority + tn_majority) if (fp_majority + tn_majority) > 0 else 0
    tpr_majority = tp_majority / (tp_majority + fn_majority) if (tp_majority + fn_majority) > 0 else 0

    fp_minority = np.sum((predictions == 1) & (true_labels == 0) & (dataset[sensitive_attribute] == minority_class))
    tn_minority = np.sum((predictions == 0) & (true_labels == 0) & (dataset[sensitive_attribute] == minority_class))
    tp_minority = np.sum((predictions == 1) & (true_labels == 1) & (dataset[sensitive_attribute] == minority_class))
    fn_minority = np.sum((predictions == 0) & (true_labels == 1) & (dataset[sensitive_attribute] == minority_class))
    fpr_minority = fp_minority / (fp_minority + tn_minority) if (fp_minority + tn_minority) > 0 else 0
    tpr_minority = tp_minority / (tp_minority + fn_minority) if (tp_minority + fn_minority) > 0 else 0

    aaod_value = 0.5 * (abs(fpr_minority - fpr_majority) + abs(tpr_minority - tpr_majority))

    return aaod_value

In [5]:
# Import dataset
df_dataset = pd.read_csv('dataset.csv', index_col=[0])
df_dataset.head()

df_score_1 = pd.read_csv('score_1.csv', index_col=[0])
df_score_2 = pd.read_csv('score_2.csv', index_col=[0])
df_score_3 = pd.read_csv('score_3.csv', index_col=[0])

df_eval_1 = pd.read_csv('eval_1.csv', index_col=[0])
df_eval_2 = pd.read_csv('eval_2.csv', index_col=[0])
df_eval_3 = pd.read_csv('eval_3.csv', index_col=[0])

In [6]:
# Analyze with fairness metrics

# For work authorization
spd_work_1 = spd('Work authorization',df_dataset, df_eval_1, 1, 0)
di_work_1 = di('Work authorization',df_dataset, df_eval_1, 1, 0)

print("Result 1")
print("SPD: ", spd_work_1)
print("DI: ", di_work_1)
print()

spd_work_2 = spd('Work authorization',df_dataset, df_eval_2, 1, 0)
di_work_2 = di('Work authorization',df_dataset, df_eval_2, 1, 0)

print("Result 2")
print("SPD: ", spd_work_2)
print("DI: ", di_work_2)
print()

spd_work_3 = spd('Work authorization',df_dataset, df_eval_3, 1, 0)
di_work_3 = di('Work authorization',df_dataset, df_eval_3, 1, 0)

print("Result 3")
print("SPD: ", spd_work_3)
print("DI: ", di_work_3)


Result 1
SPD:  prediction   -0.030564
dtype: float64
DI:  0.8257179397744447

Result 2
SPD:  prediction   -0.005546
dtype: float64
DI:  0.9598286811401565

Result 3
SPD:  prediction   -0.026935
dtype: float64
DI:  0.8281290658339838


In [7]:
# For work authorization
spd_work_1 = spd('Work authorization',df_dataset, df_eval_1, 1, 0)
di_work_1 = di('Work authorization',df_dataset, df_eval_1, 1, 0)

print("Result 1")
print("SPD: ", spd_work_1)
print("DI: ", di_work_1)
print()

spd_work_2 = spd('Work authorization',df_dataset, df_eval_2, 1, 0)
di_work_2 = di('Work authorization',df_dataset, df_eval_2, 1, 0)

print("Result 2")
print("SPD: ", spd_work_2)
print("DI: ", di_work_2)
print()

spd_work_3 = spd('Work authorization',df_dataset, df_eval_3, 1, 0)
di_work_3 = di('Work authorization',df_dataset, df_eval_3, 1, 0)

print("Result 3")
print("SPD: ", spd_work_3)
print("DI: ", di_work_3)

Result 1
SPD:  prediction   -0.030564
dtype: float64
DI:  0.8257179397744447

Result 2
SPD:  prediction   -0.005546
dtype: float64
DI:  0.9598286811401565

Result 3
SPD:  prediction   -0.026935
dtype: float64
DI:  0.8281290658339838


In [8]:
# # LIME
# from sklearn.compose import ColumnTransformer
# from sklearn.pipeline import Pipeline

# d = pd.read_csv('dataset.csv')
# e = pd.read_csv('eval_1.csv')

# d = d.fillna('N/A', inplace=True)

# model = LogisticRegression()

# # Train your model
# X = d
# y = e['prediction']
# model.fit(X, y)

# explainer = LimeTabularExplainer(X.values,
#                                 feature_names=X.columns,
#                                 class_names=['eval'],
#                                 verbose=True,
#                                 mode='regression')

# # Choose a random instance for explanation
# i = np.random.randint(0, X.shape[0])
# exp = explainer.explain_instance(X.values[i], model.predict, num_features=5)

# # Visualize the explanation
# exp.show_in_notebook(show_table=True)

In [28]:
# SHAP
# import xgboost

# # Train your model
# X = df_dataset
# # X = X.fillna(0, inplace=True)
# X = X.astype('float')
# y = df_eval_1['prediction']
# X_encoded = pd.get_dummies(X)
# model = xgboost.XGBClassifier().fit(X_encoded, y)

In [23]:
# compute SHAP values
explainer = shap.Explainer(model, X_encoded)
shap_values = explainer(X)
shap.plots.beeswarm(shap_values)

TypeError: Cannot cast array data from dtype('O') to dtype('float64') according to the rule 'safe'